# Multi-Layer Perceptron with MNIST handwritten digits classification 

## 1. Module Import

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import torch
from torchvision import datasets
import torchvision.transforms as transforms

## 2. 딥러닝 모델을 설계할 때 활용하는 장비 확인

In [2]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print('Using PyTorch version:', torch.__version__, ' Device:', device)

Using PyTorch version: 1.11.0  Device: cpu


## 3. MNIST 데이터 다운로드 (Train data와 Test data 분리하기)

In [3]:
BATCH_SIZE = 32

train_data = datasets.MNIST('./data', train=True, download=True, transform=transforms.ToTensor())
test_data = datasets.MNIST('./data', train=False, download=True, transform=transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=False)
test_loader = torch.utils.data.DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=False)


## 4. 첫번째 batch 데이터의 크기와 타입을 확인하기

In [4]:
for (X_train, y_train) in train_loader:
    print('X_train:', X_train.size(), 'type:', X_train.type())
    print('y_train:', y_train.size(), 'type:', y_train.type())
    break

X_train: torch.Size([32, 1, 28, 28]) type: torch.FloatTensor
y_train: torch.Size([32]) type: torch.LongTensor


# Loss 구하기 
   forward propagation을 계산하는 함수 구현하기
   
   1) input layer (입력층), hidden layer (은닉층), output layer (출력층) 으로 이루어진 모델을 이용

   2) 하나의 hidden layer (은닉층)만 이용 - 은닉층의 개수는 100개로 하세요

   3) 모든 것은 tensor 계산으로만 할 것!! 


## ReLU, one_hot_encoding, softmax, cross_entropy 구하기

아래 코드는 각 함수가 맞는지 확인하기 위해서 만든 임의의 값입니다. 
각 함수가 작동을 잘하는지 확인해 보세요

In [5]:
test_data = torch.tensor([[1,-2,-4, 2, 5, 6, -3, -5, 0, 2],[2, -3, 4, 3, -1, -4, 3, 5, 2, -3]])
true_label = torch.tensor([5,7])
false_label = torch.tensor([1,0])

In [6]:
def ReLU_func(outputs):  #0보다 큰 값이면 자기 자신의 값을 가지고 0보다 작다면 0을 가진다.
    zero_tensor = torch.zeros(outputs.size())
    final_outputs = torch.maximum(outputs, zero_tensor)

    return final_outputs

ReLU 함수가 맞는지 test_data를 이용하여 맞추어 보자. 아래 함수의 결과는 어떻게 예상되는가?

In [19]:
ReLU_func(test_data)

tensor([[1., 0., 0., 2., 5., 6., 0., 0., 0., 2.],
        [2., 0., 4., 3., 0., 0., 3., 5., 2., 0.]])

In [20]:
def one_hot_encoding(label):  #0부터 9까지 중에 숫자를 구하는 것이므로 크키가 10이고 답이 아닌 노드들은 0으로, 답인 노드는 1로 두어 loss를 구한다.
                              #label값은 우리가 원하는 값
    #one_hot_outputs = torch.sparse.torch.eye(10)
    #one_hot_outputs = one_hot_outputs.index_select(0,label)
    encode = torch.eye(10,10)
    one_hot_outputs = encode[label]
    
    return one_hot_outputs

one_hot_encoding 함수가 맞는지 true_label, false_label을 이용하여 맞추어 보자. 아래 함수 결과는 어떻게 예상되는가?

In [21]:
# 검증을 위해 아래 4줄을 사용하면 됩니다. 
tl = one_hot_encoding(true_label)  
print(tl)
fl = one_hot_encoding(false_label)
print(fl)

# 나중을 위해서 사용될 코드 입니다. 
close_tl = tl.clone()
close_tl[[0,1],true_label] -= 0.001
close_tl[false_label] += 0.0001
print(close_tl)

tensor([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.]])
tensor([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])
tensor([[1.0000e-04, 1.0000e-04, 1.0000e-04, 1.0000e-04, 1.0000e-04, 9.9910e-01,
         1.0000e-04, 1.0000e-04, 1.0000e-04, 1.0000e-04],
        [1.0000e-04, 1.0000e-04, 1.0000e-04, 1.0000e-04, 1.0000e-04, 1.0000e-04,
         1.0000e-04, 9.9910e-01, 1.0000e-04, 1.0000e-04]])


In [22]:
def softmax(outputs):
    numerator = torch.exp(outputs - torch.max(outputs,axis=1)[0].view(-1,1))
    denominator = torch.sum(numerator, axis=1).view(-1,1)
    softmax = numerator/denominator
    
    return softmax

softmax 함수가 맞는지 test_data를 이용하여 맞추어 보자. 아래 함수의 결과는 어떻게 예상되는가?

In [23]:
a = softmax(test_data)
print(a)
torch.sum(a,axis=1)

tensor([[4.7643e-03, 2.3720e-04, 3.2102e-05, 1.2951e-02, 2.6012e-01, 7.0709e-01,
         8.7262e-05, 1.1810e-05, 1.7527e-03, 1.2951e-02],
        [2.8590e-02, 1.9264e-04, 2.1126e-01, 7.7716e-02, 1.4234e-03, 7.0868e-05,
         7.7716e-02, 5.7425e-01, 2.8590e-02, 1.9264e-04]])


tensor([1., 1.])

In [24]:
def cross_entropy(outputs, labels):
    
    return torch.sum(-1*labels*torch.log(outputs),axis=1)

cross_entropy 함수가 맞는지 test_data를 이용하여 맞추어 보자. 아래 함수의 결과는 어떻게 예상되는가? tl, fl, close_tl을 이용하여 각각의 cross entropy를 구하고 그 값이 맞는지 확인하세요 

In [25]:
ideal_result = cross_entropy(close_tl, tl)
non_ideal_result = cross_entropy(close_tl,fl)

print(ideal_result)
print(non_ideal_result)

true_result = cross_entropy(a,tl)
false_result = cross_entropy(a,fl)

print(true_result)
print(false_result)


tensor([0.0009, 0.0009])
tensor([9.2103, 9.2103])
tensor([0.3466, 0.5547])
tensor([8.3466, 3.5547])


In [83]:
def forward_pass(train_loader):
    for batch_idx, (image, label) in enumerate(train_loader):
        
        # 이미지와 label를 1개로 만듬
        image = image[0]
        label = label[0]
        
        image = image.view(-1, 28 * 28)
        print(image.size())
        
        # Weight(가중치)를 초기화 (torch rand 함수 이용, 도중에 빼기 0.5를 하여 함수값이 -0.5~0.5 사이로 만드세요)
        W_ih = torch.rand(28*28, 100, dtype = torch.float) - 0.5
        W_ho = torch.rand(100, 10, dtype = torch.float) - 0.5
        
        # Forward propagration 계산하기.
        
        # 첫번째 Layer의 값
        outputs = image.matmul(W_ih)
        
        # 결과 값(outputs)을 ReLU 함수에 적용하기
        outputs = ReLU_func(outputs)
        
        # 출력 layer 계산 하기
        outputs = outputs.matmul(W_ho)
        
        # softmax 하기
        softmaxed_outputs = softmax(outputs)
        pred = torch.argmax(softmaxed_outputs,dim=1)
        print(pred)
        train_correct = (pred == y_train).float().mean()
        print(train_correct)
    
        
             
        # label 값을 one_hot 형태로 변환하기
        expected_outputs = one_hot_encoding(label)
        
        #loss 값 구하기
        loss = cross_entropy(softmaxed_outputs, expected_outputs)
        print(loss)
        loss = (loss - BATCH_SIZE).pow(2).sum()
        print(loss.item())
        
        break

In [81]:
forward_pass(train_loader) # 결과물이 맞다면 아래와 같은 값이 나오게 됩니다. 단, 2번째 줄의 값은 변동 가능합니다.

torch.Size([1, 784])
tensor([5])
0.0625
tensor([0.1331])
1015.5020751953125
